<a href="https://colab.research.google.com/github/alejandra2826/Alejandra-Trujillo/blob/main/Practica_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**PRACTICA 4**

* **Nombre:** Alejandra Elizabeth Trujillo Navarro
* **e-mail:** alejandra.trujillo2826@alumnos.udg.mx

 # **MODULES**

In [14]:
import math
import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import cauchy
from scipy.stats import levy_stable

import panel as pn
import panel.widgets as pnw

pn.extension('plotly')
from scipy.spatial import distance

#**CLASSES**

In [ ]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # rotate vector
    def rotated(self, angle):
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

* Funciones que generen trayectorias tipo Brownian Motion (BM)
* Correlated Random
* Walk (CRW) y Lévy Flight (LF).
* Cada una de las funciones deberá tomar como parámetros el numero de pasos, la
velocidad y posición inicial.
* Además la funciones para CRW y LF deberán tomar como parámetro el
coeficiente para la distribución Cauchy.

In [ ]:
######################################################################
# Brownian Motion Trajectory
######################################################################
def bm_2d(n_steps=1000, speed=5, s_pos=[0,0]):
  """
  Arguments:
    n_steps: number of steps the Brownian Trajectory will take -> int
    speed: speed of the trajectory or step size -> int
    s_pos: initial position -> [x,y] list
  Returns:
    BM_2d_df: DataFrame with x,y points of the full trajectory
  """

  # Init velocity vector
  velocity = Vec2d(speed, 0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])

    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  return BM_2d_df

In [ ]:
def crw(n_steps=1000, velocidad=5, s_pos=[0, 0], exponente_CRW=0.5, cauchy_coefficient=0.5):

    CRW_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])
    CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    # Inicializar el vector de velocidad
    velocidad_vector = np.array([velocidad, 0])

    for i in range(n_steps - 1):
        angulo_giro = np.random.standard_cauchy() * exponente_CRW
        matriz_rotacion = np.array([[np.cos(angulo_giro), -np.sin(angulo_giro)],
                                    [np.sin(angulo_giro), np.cos(angulo_giro)]])

        velocidad_vector = np.dot(matriz_rotacion, velocidad_vector)

        # Actualizar posición
        cauchy_noise = cauchy_coefficient * np.random.standard_cauchy()
        nueva_posicion = np.array([CRW_df.x_pos[i] + velocidad_vector[0] + np.sqrt(1 - cauchy_coefficient**2) * cauchy_noise,
                                   CRW_df.y_pos[i] + velocidad_vector[1] + np.sqrt(1 - cauchy_coefficient**2) * cauchy_noise])
        temp_df = pd.DataFrame([{'x_pos': nueva_posicion[0], 'y_pos': nueva_posicion[1]}])
        CRW_df = pd.concat([CRW_df, temp_df], ignore_index=True)

    return CRW_df
result_crw = crw(n_steps=1000, cauchy_coefficient=0.5)

# Imprime el resultado
print(result_crw)


           x_pos        y_pos
0              0            0
1       3.342013     3.609823
2        7.81423     5.991158
3      12.373002    11.431401
4      16.724121    16.517761
..           ...          ...
995 -4749.439835 -4207.462419
996 -4747.303893 -4202.847525
997 -4746.883928 -4197.179152
998 -4749.995379 -4193.335318
999 -4751.126946 -4187.784219

[1000 rows x 2 columns]


* Por último, la función para LF también deberá también aceptar como parámetro
el exponente Lévy (alpha).

In [20]:
def levy_flight_alternative(n_steps=100, init_speed=10, start_pos=[0, 0], crw_exponent=0.5, alpha=1.5, beta=0, loc=3.0, cauchy_coefficient=0.5):
    levy_df = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos': start_pos[0], 'y_pos': start_pos[1]}])
    levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    for i in range(n_steps - 1):
        # Select turn angle and step size
        turn_angle = np.random.normal(scale=crw_exponent)
        step_size = cauchy_coefficient * np.random.standard_cauchy() + loc

        # Update position
        new_position = [levy_df.x_pos[i] + np.cos(turn_angle) * step_size,
                        levy_df.y_pos[i] + np.sin(turn_angle) * step_size]

        temp_df = pd.DataFrame([{'x_pos': new_position[0], 'y_pos': new_position[1]}])
        levy_df = pd.concat([levy_df, temp_df], ignore_index=True)

    return levy_df

result_levy = levy_flight_alternative(n_steps=100, cauchy_coefficient=0.5)

# Imprime el resultado
print(result_levy)


         x_pos      y_pos
0            0          0
1      2.49021   0.123356
2     3.126694    0.25809
3     6.232184  -1.351867
4     6.985551   -0.96294
..         ...        ...
95  221.572867  36.899419
96  226.802147  36.963637
97  229.297349  36.958679
98  232.979013   36.31057
99  234.937311  37.765987

[100 rows x 2 columns]


* **Funciones para calcular las métricas Path length and Mean Squared Displacement.**

# **FUNCTIONS PATH LENGTH**


In [38]:
def path_length_alternative(trajectory):
    dis_aux = []
    for i in range(1, trajectory.shape[0]):
        dis_aux.append(distance.euclidean(trajectory.iloc[i - 1], trajectory.iloc[i]))
    return pd.DataFrame(np.cumsum(dis_aux), columns=['PL'])

#**FUNCTION MEAN SQUARED DISPLACEMENT.**

In [24]:
def euclidean_distance_alternative(q, p):
    # Utilizar la función numpy para realizar los cálculos de manera más concisa
    distance = np.linalg.norm(np.array(q) - np.array(p))
    return distance

In [25]:
def calculate_msd(trajectory_data, window_size=50):
    """
    Calculate the Mean Squared Displacement (MSD) for a given trajectory over a specified window size.

    Parameters:
        trajectory_data (pd.DataFrame): DataFrame containing the trajectory points.
        window_size (int): The size of the window to calculate the MSD over.

    Returns:
        pd.DataFrame: A DataFrame containing the MSD values.
    """
    msd_values = []
    for i in range(window_size, len(trajectory_data)):
        squared_displacements = [
            distance.euclidean(trajectory_data.iloc[j-window_size], trajectory_data.iloc[j])**2
            for j in range(window_size, i+1)
        ]
        mean_squared_displacement = np.mean(squared_displacements)
        msd_values.append(mean_squared_displacement)

    return pd.DataFrame(msd_values, columns=['MSD'])

#**DASH BOARD**

In [39]:
# Extensión de Panel
pn.extension()

# Definición de variables globales
global traj_global_df
traj_global_df = None

# Definición de widgets
trajectory_type = pn.widgets.ToggleGroup(name='Panel', options=['BM', 'CRW', 'LF'], behavior="radio", width=320, align='center')
n_steps = pn.widgets.IntSlider(name='Number of steps:', start=10, end=1000, value=500, width=320, align='center')
x_pos = pn.widgets.IntInput(name='xInitPos', value=0, width=150)
y_pos = pn.widgets.IntInput(name='yInitPos', value=0, width=150)
speed = pn.widgets.IntSlider(name='Speed', start=1, end=15, value=5, width=320, align='center')
metric = pn.widgets.Select(name='Metric', options=['PL', 'MSD'], width=320, align='center')
crw_exponent = pn.widgets.FloatInput(name='Cauchy coefficient', width=320, visible=False, align='center', step=1e-1, start=1e-1, end=1, value=0.5)
alpha = pn.widgets.FloatInput(name='Alpha', visible=False, width=320, align='center', step=1e-1, start=1e-1, end=2, value=1.5)



In [40]:
# Creación del dashboard
dashboard = pn.Row(
    pn.WidgetBox(pn.pane.Markdown("# Panel options", align='center'),
                 trajectory_type,
                 n_steps,
                 pn.Row(x_pos, y_pos, align='center'),
                 speed,
                 metric,
                 crw_exponent,
                 alpha,
                 width=380,
                 height=500, align='center'),
    pn.Column(plot_traj, width=500, height=500, align='center'),
    pn.Column(plot_metric, width=500, height=500, align='center'),
    margin=100
)

# Mostrar el dashboard
dashboard.servable()

Row(margin=100)
    [0] WidgetBox(align='center', height=500, sizing_mode='fixed', width=380)
        [0] Markdown(str, align='center')
        [1] RadioButtonGroup(align='center', name='Panel', options=['BM', 'CRW', 'LF'], value='BM', width=320)
        [2] IntSlider(align='center', end=1000, name='Number of steps:', start=10, value=500, width=320)
        [3] Row(align='center')
            [0] IntInput(name='xInitPos', width=150)
            [1] IntInput(name='yInitPos', width=150)
        [4] IntSlider(align='center', end=15, name='Speed', start=1, value=5, width=320)
        [5] Select(align='center', name='Metric', options=['PL', 'MSD'], value='PL', width=320)
        [6] FloatInput(align='center', end=1, name='Cauchy coefficient', start=0.1, value=0.5, visible=False, width=320)
        [7] FloatInput(align='center', end=2, name='Alpha', start=0.1, value=1.5, visible=False, width=320)
    [1] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)
    [2] Column(align='center', height=500, sizing_mode='fixed', width=500)
        [0] ParamFunction(function, _pane=Plotly, defer_load=False)